# Visualization of information - Project 1
## Grau en Ciència i Enginyeria de Dades
### Laia Royo and Matilde Simoes


In [ ]:
# first, we import all the necessary libraries for the project
import pandas as pd
import altair as alt
import glob
from vega_datasets import data
import requests

# to execute the following cells you should have all the files included in the zip uploded in collab !
# it is recomendable to execute JUST the part 2, as this first part takes around 10 minuts and changes are saved in CSVs.

In [ ]:
!pip install geopy

from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Part 1 - Cleaning data



In the first place, to obtain a complete database, we downloaded all the GunViolence databases from the last 6 years and merged them.

In [ ]:
"""
gun_data = pd.read_csv('GunViolenceAllYears.csv')

# we create a list with the document to merge
list_csvs = ['GunViolence2018.csv','GunViolence2019.csv','GunViolence2020.csv','GunViolence2021.csv','GunViolence2022.csv','GunViolence2023.csv']

# we combine them
csv_combined = pd.concat([pd.read_csv(f) for f in list_csvs])

# finally we save the documents in a csv
csv_combined.to_csv('GunViolenceAllYears.csv')
"""

"\ngun_data = pd.read_csv('GunViolenceAllYears.csv')\n\n# we create a list with the document to merge\nlist_csvs = ['GunViolence2018.csv','GunViolence2019.csv','GunViolence2020.csv','GunViolence2021.csv','GunViolence2022.csv','GunViolence2023.csv']\n\n# we combine them\ncsv_combined = pd.concat([pd.read_csv(f) for f in list_csvs])\n\n# finally we save the documents in a csv\ncsv_combined.to_csv('GunViolenceAllYears.csv')\n"

For the next part we used OpenRefine and Python and we have taken the following steps:

1. **Data Import and Parsing**: We correctly imported the dataset, ensuring variables were properly separated into distinct columns, avoiding issues with mixed data formats.

2. **Data Type Standardization**: We transformed variables to their appropriate data types. For instance, the columns Victims Killed, Victims Injured, Suspects Killed, and Suspects Injured were converted to numerical types, while Incident Date was standardized to a date format.

3. **Missing Data Handling**: We checked for missing or blank values in the dataset but found none.

4. **Outlier Detection**: Using the faceting feature, we visualized all numerical variables to detect potential outliers. While we verified that there were no illogical values, such as negative numbers in columns related to casualties, we did find some values that were noticeably larger than others. However, upon review, these values could reasonably represent real cases.

5. **Text Standardization and Clustering**: For categorical or text-based variables, we applied clustering techniques to identify and merge inconsistent entries. For example, we discovered two incidents where the same countie was named in a different way and we corrected it.

6. **Creation of the 'State Code' Column**: A new column called 'State Code' was created to assist with future visualizations. This column contains the codes used by Altair to refer to each state, ensuring consistency across the dataset and preparing it for analysis and graphical representation. (using the dictonary below)

7. **Corrections in the 'City and County' Column**: In the provided dataset, there is a column that contains either a city or a county without any specific order. To standardize the data, we will convert the entries that represent cities into counties. Using the *fuzzywizzy* library.

8. **Creation of the 'State Population' and 'County Population' Columns**: Two new columns were created to include the population of each state and county, simplifying future work when generating visualizations per citizen. This was achieved using external datasets from EU census. The population of states data is from 2019 and the counties' data from 2023.

### Code in Collab

Step 7 - Corrections in the 'City and County' Column

In [ ]:
def get_place_type(name):
  """Given a name says if is a city or a county"""
  url = "https://nominatim.openstreetmap.org/search"
  headers = {
      'User-Agent': 'GeoVisualizer/1.0 (tu@email.com)'
  }
  params = {
      "q": name,
      "format": "json",
      "addressdetails": 1,
      "limit": 1
  }
  response = requests.get(url, headers=headers, params=params)

  print(f"Resposta del servidor: {response.text}")

  try:
      data = response.json()
      if data:
          if "address" in data[0]:
              if "city" in data[0]["address"]:
                  return "city"
              elif "county" in data[0]["address"]:
                  return "county"
              else:
                  return "unknown"
          else:
              return "No s'ha trobat cap resultat."
      else:
          return "No s'ha trobat cap resultat."
  except ValueError:
      return "Error en el format de la resposta."

def get_city_county(city_name):
  """Donada una ciutat ens dona el seu county"""
  url = "https://nominatim.openstreetmap.org/search"
  headers = {
      'User-Agent': 'GeoVisualizer/1.0 (tu@email.com)'  # Canvia per a la teva aplicació
  }
  params = {
      "q": city_name,
      "format": "json",
      "addressdetails": 1,
      "limit": 1
  }
  response = requests.get(url, headers=headers, params=params)

  print(f"Resposta del servidor: {response.text}")

  try:
      data = response.json()
      if data:
          if "address" in data[0]:
              county = data[0]["address"].get("county", "County no trobat")
              return county
          else:
              return "No s'ha trobat cap resultat."
      else:
          return "No s'ha trobat cap resultat."
  except ValueError:
      return "Error en el format de la resposta."

In [ ]:
gun_data = pd.read_csv('cleandata_GunViolenceAllYears.csv')

In [ ]:
# we extract unique values from 'City Or County' column
unique_cities_list = gun_data['City Or County'].unique().tolist()

# and make a dictionary to convert each city to county
place_dict = {}
for c in unique_cities_list:
  if get_place_type(c) == 'city':
      place_dict[c] = get_city_county(c)
  else:
      place_dict[c] = c

Resposta del servidor: [{"place_id":342666654,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"relation","osm_id":182130,"lat":"41.4996574","lon":"-81.6936772","class":"boundary","type":"administrative","place_rank":16,"importance":0.6952330131061538,"addresstype":"city","name":"Cleveland","display_name":"Cleveland, Cuyahoga County, Ohio, United States","address":{"city":"Cleveland","county":"Cuyahoga County","state":"Ohio","ISO3166-2-lvl4":"US-OH","country":"United States","country_code":"us"},"boundingbox":["41.3906280","41.6044360","-81.8790937","-81.5327440"]}]
Resposta del servidor: [{"place_id":342666654,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"relation","osm_id":182130,"lat":"41.4996574","lon":"-81.6936772","class":"boundary","type":"administrative","place_rank":16,"importance":0.6952330131061538,"addresstype":"city","name":"Cleveland","display_name":"Cleveland, Cuyahoga County, Ohi

In [ ]:
# we load the external csv with county population data
population = pd.read_csv('pop_and_code_data.csv')
gun_data = pd.read_csv('cleandata_GunViolenceAllYears.csv')
population['City Or County'] = population['CTYNAME']
population.head()

,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,City Or County
0,1,1,Alabama,Autauga County,4,59726,Autauga County
1,1,3,Alabama,Baldwin County,4,246531,Baldwin County
2,1,5,Alabama,Barbour County,4,24700,Barbour County
3,1,7,Alabama,Bibb County,4,21986,Bibb County
4,1,9,Alabama,Blount County,4,59516,Blount County


In [ ]:
# and convert the values of 'City Or County' column to counties
for c in gun_data['City Or County']:
  gun_data['City Or County'] = gun_data['City Or County'].replace(c, place_dict[c])

In [ ]:
# because of having diferent sources, some names don't match so we apply clustering to find the most similar values from both dataset
def create_match_dict(reference_list, target_list):
    match_dict = {}
    for target in target_list:
        best_match, score = process.extractOne(target, reference_list)
        match_dict[target] = best_match  # Store the best match in the dictionary
    return match_dict

reference_list = list(population['City Or County'].unique())  # unique values from population
target_list = list(gun_data['City Or County'].unique())  # unique values from gun_data

match_dict = create_match_dict(reference_list, target_list)

gun_data['City Or County Matched'] = gun_data['City Or County'].map(match_dict)

# now, we can merge both datasets
gun_data = gun_data.merge(population, left_on='City Or County Matched', right_on='City Or County', how='left')

   Incident ID         Incident Date State  State Code  City Or County_x  \
0      1289686  2018-12-31T00:00:00Z  Ohio          39   Cuyahoga County   
1      1288376  2018-12-29T00:00:00Z  Ohio          39  County no trobat   
2      1288376  2018-12-29T00:00:00Z  Ohio          39  County no trobat   
3      1288376  2018-12-29T00:00:00Z  Ohio          39  County no trobat   
4      1288376  2018-12-29T00:00:00Z  Ohio          39  County no trobat   

            Address  Victims Killed  Victims Injured  Suspects Killed  \
0  4462 Denison Ave               3                2                0   
1     904 S Main St               1                3                0   
2     904 S Main St               1                3                0   
3     904 S Main St               1                3                0   
4     904 S Main St               1                3                0   

   Suspects Injured  Suspects Arrested  Operations City Or County Matched  \
0                 0        

In [ ]:
gun_data.head()

Step 8 - Creation of the 'State Population' and 'County Population' Columns

In [ ]:
def code_state_column_population(state):
  "Transforms the capital letters to code state"

  state_to_code = {'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8, 'CT': 9,
      'DE': 10, 'DC': 11, 'FL': 12, 'GA': 13, 'HI': 15, 'ID': 16, 'IL': 17,
      'IN': 18, 'IA': 19, 'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24, 'MA': 25,
      'MI': 26, 'MN': 27, 'MS': 28, 'MO': 29, 'MT': 30, 'NE': 31, 'NV': 32,
      'NH': 33, 'NJ': 34, 'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38,
      'OH': 39, 'OK': 40, 'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46,
      'TN': 47, 'TX': 48, 'UT': 49, 'VT': 50, 'VA': 51, 'WA': 53, 'WV': 54,
      'WI': 55, 'WY': 56}

  return state_to_code.get(state)

# we load state population data
db_population = pd.read_csv('historical_state_population_by_year.csv', header=None)
db_population.columns = ['State', 'Year', 'Population']

db_population['State Code'] = db_population['State'].apply(code_state_column_population)

# we select the most actual data (2019)
db_population_2019 = db_population[db_population['Year'] == 2019]

gun_data['State Code'] = gun_data['State Code'].astype(str)
db_population_2019['State Code'] = db_population_2019['State Code'].astype(str)

gun_data = gun_data.merge(db_population_2019, on='State Code', how='left')
gun_data['Population_state'] = gun_data['Population']
a

<ipython-input-50-a0e0e8a589b7>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_population_2019['State Code'] = db_population_2019['State Code'].astype(str)


Finally, we save just the clumns we need.

In [ ]:
gun_data.head()
gun_data = gun_data[['Incident ID', 'Incident Date', 'State Code', 'Population_state', 'City Or County', 'State_x']]

Finally, we save the changes to the original csv.

In [ ]:
gun_data.to_csv('cleandata_GunViolenceAllYears2.csv')

The most of this steps were also followed with SchoolIncidents.csv database.

## Part 2 - Visualitation of the data


### Q1: What are the states with large number of mass shootings per citizen?

In [ ]:
gun_data = pd.read_csv("cleandata_GunViolenceAllYears2.csv")
gun_data.head()

,Unnamed: 0,Incident ID,Incident Date,State Code,Population,City Or County,State_x
0,0,1289686,2018-12-31T00:00:00Z,39,11689100,Cuyahoga County,Ohio
1,1,1288376,2018-12-29T00:00:00Z,39,11689100,County no trobat,Ohio
2,2,1287286,2018-12-28T00:00:00Z,29,6137428,Saint Charles County,Missouri
3,3,1284586,2018-12-24T00:00:00Z,22,4648794,Lutcher,Louisiana
4,4,1283229,2018-12-23T00:00:00Z,29,6137428,Sangamon County,Missouri


In [ ]:
# Group by 'State Code' and count 'Incident ID'
state_incidents = gun_data.groupby('State Code')['Incident ID'].count().reset_index(name='Incident Count')

# Merge back with the original data to retain the 'Population_x' column
state_incidents = state_incidents.merge(gun_data[['State Code', 'Population', 'State_x']].drop_duplicates(), on='State Code', how='left')

In [ ]:
state_incidents['Incidents per Citizen'] = state_incidents['Incident Count'] / state_incidents['Population'] *1000000
state_incidents = state_incidents[state_incidents['State Code'] != 11]

In [ ]:
# Sort and get the top 10 states by per capita rate
top_10_states_per_citizen = state_incidents.sort_values(by='Incidents per Citizen', ascending=False).head(10)

# Create the chart
bar_chart_per_citizen = alt.Chart(top_10_states_per_citizen).mark_bar(color='steelblue').encode(
    x=alt.X('Incidents per Citizen:Q', title='Incidents per 1M Citizens'),
    y=alt.Y('State_x:N', sort='-x', title='State'),
    tooltip=['State:N', 'Incident Count:Q', 'Incidents per 1M Citizens:Q']
).properties(
    width=700,
    height=500,
    title='Gun Violence Incidents per Citizen by State'
)

bar_chart_per_citizen

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Q2: How is the number of mass shootings per citizen distributed across the different counties in the US? And across states?

**Mass shootings distributed across the different counties**

In [ ]:
# read and show data
gun_data = pd.read_csv("cleandata_GunViolenceAllYears2.csv")

gun_data.head()

,Unnamed: 0,Incident ID,Incident Date,State Code,Population,City Or County,State_x
0,0,1289686,2018-12-31T00:00:00Z,39,11689100,Cuyahoga County,Ohio
1,1,1288376,2018-12-29T00:00:00Z,39,11689100,County no trobat,Ohio
2,2,1287286,2018-12-28T00:00:00Z,29,6137428,Saint Charles County,Missouri
3,3,1284586,2018-12-24T00:00:00Z,22,4648794,Lutcher,Louisiana
4,4,1283229,2018-12-23T00:00:00Z,29,6137428,Sangamon County,Missouri


In [ ]:
# NO VA PORQUE NO TENEMOS HECHO BIEN LO DE LOS COUNTIES
gun_data['City Or County'] = gun_data['City Or County'].map(place_dict)
gun_data['id'] = gun_data['City Or County']

# Agrupar los incidentes por estado y condado
county_incidents = gun_data.groupby('State Code').size().reset_index(name='Incident Count')

# Usar directamente la URL en Altair
counties = "https://raw.githubusercontent.com/vega/vega-datasets/master/data/us-10m.json"

county_map = alt.Chart(alt.Data(url=counties, format={"type": "topojson", "feature": "counties"})).mark_geoshape().encode(
    color='Total Incidents:Q',  # Datos de incidentes en gradiente
    tooltip=['City Or County:N', 'Incident Count:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(county_incidents, 'City Or County', ['Total Incidents'])
).project(
    type='albersUsa'
).properties(
    width=700,
    height=500,
    title='Gun Violence Incidents by County'
)

county_map

# at last, we plot the chart


alt.Chart(...)

In [ ]:
# what could we do to make the map more understandable?
#indicating the number of incidents:

In [ ]:
# there are still some dificulties tu understand. some text is on another and it is diffcult to read

**Mass shootings distributed across the different counties**

In [ ]:
state_incidents.head()

,State Code,Incident Count,Population,State_x,Incidents per Citizen
0,1,423,4903185,Alabama,86.270455
1,2,4,731545,Alaska,5.467880
2,4,41,7278717,Arizona,5.632861
3,5,155,3017804,Arkansas,51.361851
4,6,340,39512223,California,8.604932


In [ ]:
# we create the map of EEUU
states = alt.topo_feature('https://vega.github.io/vega-datasets/data/us-10m.json', feature='states')

chart = alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('Incidents per Citizen:Q', scale=alt.Scale(scheme='blues')),
    tooltip=['State Code:N', 'Incident Count:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_incidents, 'State Code', ['Incidents per Citizen'])
).project(
    type='albersUsa'
).properties(
    width=700,
    height=500,
    title='Gun Violence Incidents by State'
)

# at last, we plot the chart
chart

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Q3: Are mass shootings correlated with gun violence incidents in schools?

In [ ]:
# first, we read specific data for this visualitation
school_data = pd.read_csv('cleandata_SchoolIncidence.csv')
gun_data = pd.read_csv('cleandata_GunViolenceAllYears.csv')
school_data['Type'] = 'School'
gun_data['Type'] = 'Gun'
school_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cleandata_SchoolIncidence.csv'

In [ ]:
school_data = school_data[['Incident ID', 'Incident Date', 'State Code', 'Type']]
gun_data = gun_data[['Incident ID', 'Incident Date', 'State Code', 'Type']]
incidents_data = pd.concat([school_data, gun_data])

In [ ]:
# we converted the 'Incident Date' column to datetime
incidents_data['Incident Date'] = pd.to_datetime(incidents_data['Incident Date'])

incidents_data = incidents_data[incidents_data['Incident Date'].dt.year.isin([2023])]

incidents_data['Month'] = incidents_data['Incident Date'].dt.month

incidents_data.head()

**Evolution of mass shooting and gun violence by the time**

In [ ]:
# Group by the new 'Month' column and 'Type' of incident
incidents_over_time = incidents_data.groupby(['Month', 'Type']).size().reset_index(name='Count')

# Plot
chart = alt.Chart(incidents_over_time).mark_line().encode(
    x='Month:T',  # Use 'Month' as the x-axis (time axis)
    y='Count:Q',  # Count of incidents on the y-axis
    color='Type:N'  # Different colors for each incident type
).properties(
    title='Gun vs. School Incidents Over Time by Month',
    width=800,
    height=400
)

chart

**Correlation between gun violence and mass shooting**

In [ ]:
state_counts = incidents_data.groupby(['State Code', 'Type']).size().unstack(fill_value=0).reset_index()

# Compute correlation between 'School' and 'Gun'
correlation = state_counts.corr().loc['School', 'Gun']

# Prepare data for visualization
state_counts_melted = state_counts.melt(id_vars='State Code', value_vars=['School', 'Gun'],
                                        var_name='Type', value_name='Count')

# Create a scatter plot to show the relationship between School and Gun incidents by state
scatter = alt.Chart(state_counts).mark_circle(size=100).encode(
    x=alt.X('School:Q', title='Number of School Incidents'),
    y=alt.Y('Gun:Q', title='Number of Gun Incidents'),
    tooltip=['State Code', 'School', 'Gun']
)

regression_line = scatter.transform_regression(
    'School', 'Gun', method='linear'
).mark_line(color='blue')

# Layer the scatter plot and regression line
chart = (scatter + regression_line).properties(
    title=f'Correlation between School and Gun Incidents by State: {correlation:.2f}',
    width=600,
    height=400
)

# Show the chart
chart.display()

### Q4: How have mass shootings evolved the last years in the US?

**Yearly Evolution of Mass Shootings**



This line chart illustrates the annual trend in the number of mass shooting incidents over the years. Each data point represents the total incidents recorded for a given year, highlighting any increases or decreases in frequency over time.

In [ ]:
gun_data = pd.read_csv("cleandata_GunViolenceAllYears.csv")

# we converted the 'Incident Date' column to datetime
gun_data['Incident Date'] = pd.to_datetime(gun_data['Incident Date'])

# we extracted the year
gun_data['Year'] = gun_data['Incident Date'].dt.year

# we calculated the number of incidents per year
yearly_incidents = gun_data.groupby('Year').size().reset_index(name='Incident Count')

# we created a line graph to show the evolution of incidents by year
line_chart = alt.Chart(yearly_incidents).mark_line(point=True).encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('Incident Count:Q', title='Number of incidents'),
    tooltip=['Year', 'Incident Count']
).properties(
    width=700,
    height=500,
    title='Evolution of mass shootings in recent years in the USA'
)

line_chart

In [ ]:
gun_data = pd.read_csv("cleandata_GunViolenceAllYears.csv")

# we converted the 'Incident Date' column to datetime
gun_data['Incident Date'] = pd.to_datetime(gun_data['Incident Date'])

# we extracted the year
gun_data['Year'] = gun_data['Incident Date'].dt.year
gun_data['Month'] = gun_data['Incident Date'].dt.month

# we calculated the number of incidents per year
yearly_incidents = gun_data.groupby('Year', 'Month').size().reset_index(name='Incident Count')

# we created a line graph to show the evolution of incidents by year
line_chart = alt.Chart(yearly_incidents).mark_line(point=True).encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('Incident Count:Q', title='Number of incidents'),
    tooltip=['Year', 'Incident Count']
).properties(
    width=700,
    height=500,
    title='Evolution of mass shootings in recent years in the USA'
)

line_chart

**Monthly Evolution of Mass Shootings Across All Years**


This line chart shows the trend of mass shootings each month for all years, with separate lines representing different years.

In [ ]:
# Convert the 'Incident Date' column to datetime format
gun_data['Incident Date'] = pd.to_datetime(gun_data['Incident Date'])

# Extract the year and month from the 'Incident Date' column
gun_data['Year'] = gun_data['Incident Date'].dt.year
gun_data['Month'] = gun_data['Incident Date'].dt.month

# Group by Year and Month, then count incidents
monthly_incidents = gun_data.groupby(['Year', 'Month']).size().reset_index(name='Incident Count')

# Create a dictionary mapping month numbers to month names
month_dict = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
              7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

# Add a new column for the month names
monthly_incidents['Month Name'] = monthly_incidents['Month'].map(month_dict)

# Create a line chart to show the monthly evolution for each year
line_chart = alt.Chart(monthly_incidents).mark_line(point=True).encode(
    x=alt.X('Month Name:O', title='Month',
            sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                  'September', 'October', 'November', 'December'],
            axis=alt.Axis(title='Month')),
    y=alt.Y('Incident Count:Q', title='Number of incidents'),
    color=alt.Color('Year:N', title='Year'),
    tooltip=['Year', 'Month Name', 'Incident Count']
).properties(
    width=800,
    height=500,
    title='Monthly Evolution of Mass Shootings in Recent Years in the USA'
)

line_chart

**Smoothed Monthly Trend of Mass Shootings by Year**


The Smoothed Line Chart smooths data to reduce the appearance of sharp fluctuations between points, making it easier to visualise an overall trend over time.

In [ ]:
# Apply moving average to smooth the trends
monthly_incidents['Smoothed Incident Count'] = monthly_incidents.groupby('Month Name')['Incident Count'].transform(lambda x: x.rolling(3, min_periods=1).mean())

# Smoothed line chart
smoothed_line_chart = alt.Chart(monthly_incidents).mark_line(point=True).encode(
    x=alt.X('Month Name:O', title='Month',
            sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                  'September', 'October', 'November', 'December']),
    y=alt.Y('Smoothed Incident Count:Q', title='Smoothed Number of Incidents'),
    color=alt.Color('Year:N', title='Year'),
    tooltip=['Year', 'Month Name', 'Smoothed Incident Count']
).properties(
    width=800,
    height=500,
    title='Smoothed Trend of Mass Shootings by Month and Year'
)

smoothed_line_chart

**Monthly Incidents of Mass Shootings by Year**


This bar chart shows the count of mass shooting incidents by month for each year. Each bar segment represents incidents for one year, allowing comparison across months within a year.

In [ ]:
bar_chart = alt.Chart(monthly_incidents).mark_bar().encode(
    x=alt.X('Month Name:O', title='Month',
            sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                  'September', 'October', 'November', 'December']),
    y=alt.Y('Incident Count:Q', title='Number of incidents'),
    color=alt.Color('Year:N', title='Year'),
    tooltip=['Year', 'Month Name', 'Incident Count']
).properties(
    width=800,
    height=500,
    title='Monthly Incidents of Mass Shootings by Year'
)

bar_chart

**Box Plot of Mass Shootings by Month**


The box plot displays the distribution of mass shooting incidents for each month, across all years. Highlights outliers, median values, and variability in incident counts per month.

In [ ]:
# Convert 'Incident Date' to datetime format if not already done
gun_data['Incident Date'] = pd.to_datetime(gun_data['Incident Date'])

# Extract year and month from the 'Incident Date'
gun_data['Year'] = gun_data['Incident Date'].dt.year
gun_data['Month'] = gun_data['Incident Date'].dt.month

# Group by year and month to count incidents
monthly_incidents = gun_data.groupby(['Year', 'Month']).size().reset_index(name='Incident Count')

# Map month numbers to names for readability
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August',
               9: 'September', 10: 'October', 11: 'November', 12: 'December'}
monthly_incidents['Month Name'] = monthly_incidents['Month'].map(month_names)

# Now try creating the box plot again
box_plot = alt.Chart(monthly_incidents).mark_boxplot().encode(
    x=alt.X('Month Name:O', title='Month',
            sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                  'September', 'October', 'November', 'December']),
    y=alt.Y('Incident Count:Q', title='Number of incidents'),
    color=alt.Color('Month Name:N', title='Month'),
    tooltip=['Month Name', 'Incident Count']
).properties(
    width=800,
    height=500,
    title='Box Plot of Monthly Mass Shootings'
)

box_plot

**Heatmap of Mass Shootings by Month and Year**


This heatmap provides a color-coded view of the frequency of mass shooting incidents by month and year. Darker shades represent higher incident counts, allowing for easy identification of years and months with high activity.

In [ ]:
heatmap_chart = alt.Chart(monthly_incidents).mark_rect().encode(
    x=alt.X('Month Name:O', title='Month',
            sort=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                  'September', 'October', 'November', 'December']),
    y=alt.Y('Year:N', title='Year'),
    color=alt.Color('Incident Count:Q', scale=alt.Scale(scheme='blues'), title='Incident Count'),
    tooltip=['Year', 'Month Name', 'Incident Count']
).properties(
    width=800,
    height=500,
    title='Heatmap of Mass Shootings by Year and Month'
)

heatmap_chart